<a href="https://colab.research.google.com/github/ronykroy/NLP/blob/master/H20_multiClass_newsGroups_AutoML_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# v3: removed the text column in x: predictors
# TODO: apply the w2v for just the train.. then apply that leanred w2v model to the training/test set
#todo w2v take a 100 more as the lenth of the vector

In [2]:
!apt-get install default-jre
!java -version
# install Java.. because.. there is nothing in this colab VM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [3]:
! pip install h2o
# H2O too to be installed SMH

     |████████████████████████████████| 123.6MB 35kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.8-py2.py3-none-any.whl size=123708939 sha256=e6b11647e685261fd68dc55a948a17e417e12050a3d7972920adcb9adae33d61
  Stored in directory: /root/.cache/pip/wheels/f7/4c/2f/367de2996c0c2621a9e2e19624be0ea0e4b766df3cd597d916
Successfully built h2o


In [0]:
import h2o
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
h2o.init(max_mem_size="4G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphl96s7i5
  JVM stdout: /tmp/tmphl96s7i5/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmphl96s7i5/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_unknownUser_kekk9c
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [6]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

INFO:sklearn.datasets.twenty_newsgroups:Downloading 20news dataset. This may take a few minutes.
INFO:sklearn.datasets.twenty_newsgroups:Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})
df_trn, df_test = train_test_split(df, stratify = df['label'], test_size = 0.15, random_state = 11) 

In [8]:
reviews = h2o.H2OFrame(df)
reviews_train = h2o.H2OFrame(df_trn)
reviews_test = h2o.H2OFrame(df_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
reviews_train.head(2)

label,text
0,"In emarsh@hernes-sun.Eng.Sun.COM (Eric Huh? Please explain. Is there a problem because I based my morality on something that COULD be wrong? Gosh, there's a heck of a lot of stuff that I believe that COULD be wrong, and that comes from sources that COULD be wrong. What do you base your belief on atheism on? Your knowledge and reasoning? COuldn't that be wrong? MAC -- **************************************************************** Michael A. Cobb ""...and I won't raise taxes on the middle University of Illinois class to pay for my programs."" Champaign-Urbana -Bill Clinton 3rd Debate cobb@alexia.lis.uiuc.edu"
6,"Nintendo 8 bit system, power pad, light gun (zapper), 2 controllers Games: Supermario, duck hunt, power field, and wings. Asking $80."


In [0]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [0]:
# TODO: review how effective this is in removing junk \r \n etc
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [12]:
%%time
# Break reviews into sequence of words
words_train = tokenize(reviews_train["text"]) # to check against leakage here...
words_test = tokenize(reviews_test["text"]) 

CPU times: user 7.14 s, sys: 128 ms, total: 7.27 s
Wall time: 24.8 s


In [13]:
words_train.head()

C1
exodus
eng
sun
com
emarsh
hernes
sun
eng
sun
com


In [14]:
%%time
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

w2v_model = H2OWord2vecEstimator(vec_size = 200, model_id = "w2v.hex")
w2v_model.train(training_frame=words_train) # train only on  the train_words  we just got

word2vec Model Build progress: |██████████████████████████████████████████| 100%
CPU times: user 1.93 s, sys: 214 ms, total: 2.14 s
Wall time: 4min 40s


In [15]:
# Sanity check - find synonyms for the word 'president'
w2v_model.find_synonyms("president", count = 5) 

OrderedDict([('mubarak', 0.7497212886810303),
             ('yeltsin', 0.7413913011550903),
             ('stephanopoulos', 0.7083085775375366),
             ('miyazawa', 0.6971884369850159),
             ('clinton', 0.6502059102058411)])

In [0]:
# Calculate a vector for each review in each data set
review_vecs_train = w2v_model.transform(words_train, aggregate_method = "AVERAGE")
review_vecs_test = w2v_model.transform(words_test, aggregate_method = "AVERAGE")

In [17]:
review_vecs_train

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200
-0.158377,0.0942881,-0.0483496,-0.0180819,-0.0351723,-0.144311,0.0447766,-0.0350037,-0.0143421,0.0839209,0.0427587,-0.00205427,0.0902803,-0.0264904,-0.0308397,0.0291451,0.0372268,0.0310375,0.107697,0.0520165,-0.0764475,-0.100616,0.0621047,0.0925142,0.0186486,-0.0758172,0.0823149,-0.072319,-0.180096,-0.0634054,0.00225585,0.0287955,0.00282659,-0.118607,-0.027938,0.0508658,0.0385519,-0.083683,0.0673957,0.0528579,-0.0299388,0.136309,0.0364779,0.0228412,0.0779619,-0.049085,0.0378171,-0.029006,-0.0103565,-0.0106277,-0.068934,0.138639,-0.11138,0.0349748,0.055441,0.0998898,-0.0433927,0.00742945,-0.0174346,-0.0359516,-0.0116652,-0.017277,-0.0299007,-0.00572967,-0.0221933,0.0234234,0.055083,-0.00682111,0.0647923,0.019742,-0.0125749,-0.0813432,0.00199735,0.0597675,-0.0511741,-0.0943008,0.069585,-0.0180594,-0.073225,-0.0993245,-0.157401,0.0197472,0.0730079,0.0555541,0.0656919,0.0261537,0.0796379,-0.119858,-0.0052979,0.0147112,-0.119757,0.0303199,0.112806,0.0563812,0.0742114,0.0561358,0.0644193,0.0557822,-0.0412288,-0.0638677,0.0762278,-0.130016,0.0436997,0.040398,0.0611341,0.0821323,0.098043,-0.0606368,-0.0652929,-0.0289674,0.0644855,-0.00522402,0.0748285,0.101576,0.113605,0.0500773,0.0121333,-0.135094,0.0124906,0.0777823,-0.0625292,-0.0797545,-0.0314803,0.0854787,-0.0304192,0.0387167,-0.0753992,-0.0451342,0.08937,0.161424,0.0273829,-0.0853622,-0.0582526,0.082807,0.0724099,0.0726295,0.126067,0.0506533,0.00373752,-0.0895739,0.0109762,0.0365187,0.029136,0.0329148,-0.155535,0.0120959,0.0305222,-0.0649331,0.051177,0.0773834,0.0150175,-0.0503769,0.0602404,-0.0296882,-0.0590036,0.0608915,-0.116041,0.0995201,0.0642689,-0.0958357,-0.0348842,-0.0442537,-0.0259808,0.00995596,0.0138379,-0.0213475,0.168464,-0.0120742,-0.0504924,-0.0635328,-0.038213,-0.120913,-0.0332195,-0.0329549,-0.155386,0.0233363,-0.221976,0.0948118,-0.129662,0.0822599,-0.0571596,0.0195191,0.140683,-0.0220142,-0.0711455,-0.0608414,0.0453981,0.0454547,-0.156911,0.0256244,-0.0888258,-0.0795334,0.087303,-0.165294,-0.080702,-0.139045,0.0278193,0.0207571,0.0240474,0.101919
0.0664115,0.0755174,-0.0105883,-0.00751137,0.159169,-0.05036,0.104538,0.0835489,0.0268142,0.023108,0.0438652,-0.0214539,-0.00669538,-0.0570713,0.0498712,-0.0848482,-0.188714,0.0625769,0.000620642,0.043261,-0.0710653,0.0306171,0.0800802,0.0507233,-0.0673842,-0.147737,0.0553315,-0.153421,0.00153268,-0.0351984,-0.0541465,-0.0563368,0.0415996,-0.0401976,0.0128446,-0.0404825,0.032298,-0.0523354,-0.039835,0.0749611,-0.0579042,-0.0378207,0.019768,0.0678576,0.0447483,-0.0735699,-0.088735,0.0154369,-0.0439075,-0.0319716,0.0430231,0.159598,0.114735,-0.00422784,0.0620229,-0.00250155,-0.0927934,0.10074,0.0248042,0.129464,0.0380502,-0.129689,-0.206999,-0.116441,-0.0786592,0.096709,-0.0759541,0.0901081,0.0794012,-0.176557,-0.0477503,-0.0320803,-0.00638589,0.0876108,-0.101558,-0.0145571,0.108868,0.00605792,-0.0953846,0.00904962,-0.0266224,-0.0194396,0.000494304,0.110467,0.122193,-0.121729,-0.0663041,-0.0109973,-0.0305815,-0.0957564,0.0231727,-0.0690083,-0.0181223,-0.0239139,-0.0542136,-0.0311468,0.00822399,-0.11357,0.121809,0.0152614,0.0179474,-0.128914,0.0

In [0]:
# Add aggregated word embeddings 
ext_reviews_train = reviews_train.cbind(review_vecs_train)
ext_reviews_test = reviews_test.cbind(review_vecs_test)
# rename may be... all 'reviews' to 'news'
# ext becaus ewe are extending the reviews df with the vectors

In [0]:
ext_reviews_train["label_f"]  = ext_reviews_train["label"] .asfactor()


In [0]:
ext_reviews_test["label_f"]  = ext_reviews_test["label"] .asfactor() # need this too for the conf matrix

In [0]:
# Splitting to test train.. along with the review vecs is difficult without the 'Train' boolean column
# hence redoing it from that stage..
# so that we'll be able to do this following cell

In [0]:
#ext_train = ext_reviews[ext_reviews["Train"] == "True"]
#ext_test = ext_reviews[ext_reviews["Train"] == "False"] 
# these are not needed now.. since these were already done...

In [0]:
#review_vecs.names
# had a 100 cols
# may be more would give a better model ..?

In [24]:
reviews.shape[0]

12291

In [0]:
from h2o.automl import H2OAutoML

In [0]:
x = ext_reviews_train.columns

In [0]:
# x= [x for x in ext_train.columns if x not in ['label', 'Train']]

In [28]:
print(x)

['label', 'text', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'C101', 'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C108', 'C109', 'C110', 'C111', 'C112', 'C113', 'C114', 'C115', 'C116', 'C117', 'C118', 'C119', 'C120', 'C121', 'C122', 'C123', 'C124', 'C125', 'C126', 'C127', 'C128', 'C129', 'C130', 'C131', 'C132', 'C133', 'C134', 'C135', 'C136', 'C

In [0]:
# Identify predictors and response
x = ext_reviews_train.columns
x = [x for x in ext_reviews_train.columns if x not in ['label', 'Train','text','label_f']] # remove columns that can leak data
y = "label_f"
# other_cols = ""
# x.remove(y)

# For binary classification, response should be a factor
#train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()
# Already done

In [30]:
print(x)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'C101', 'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C108', 'C109', 'C110', 'C111', 'C112', 'C113', 'C114', 'C115', 'C116', 'C117', 'C118', 'C119', 'C120', 'C121', 'C122', 'C123', 'C124', 'C125', 'C126', 'C127', 'C128', 'C129', 'C130', 'C131', 'C132', 'C133', 'C134', 'C135', 'C136', 'C137', 'C138', 'C1

In [0]:
%%time
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)#, max_runtime_secs =1200)
aml.train(x=x, y=y, training_frame=ext_reviews_train)

AutoML progress: |███████████████████████████████████████████████████████

In [0]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [0]:
#aml.leader.confusion_matrix()
aml.leader.cross_validation_metrics_summary

In [0]:
#ext_reviews_test.columns

In [0]:
aml.leader.confusion_matrix(ext_reviews_test) # 

In [0]:
con_mat = aml.leader.confusion_matrix(ext_reviews_test).as_data_frame()

In [0]:
con_mat.iloc[19][19] #

In [0]:
con_mat.iloc[19][21] #

In [0]:
con_mat.iloc[0:20,0:20]

In [0]:
np.sum(np.diag(con_mat.iloc[0:20,0:20]))/(con_mat.iloc[0:20,0:20].values.sum()) # sum of diagnoal elements/ sum of all in the matrix # 38733477 & SO 53762743

### before making w2v exlusively from train data set: 0.6607298001737619
### after making w2v exlusively from train data set: 0.6395759717314488

In [0]:
# getting the absolute accuracy  as above

In [0]:
con_mat.iloc[0:20,0:20].values.sum()

In [0]:
ext_test.shape # hmmmm

In [0]:
#ext_test.head(2)
#ext_test.col_names

In [0]:
con_mat.to_csv('confusion_matrix.csv')

In [0]:
aml.leader.hit_ratio_table().as_data_frame() # SO 53762743

In [0]:
#train.label.head()
# easier to get pd data and cast it to h2o frame
df.head(2)

In [0]:
random_news  = h2o.H2OFrame(df.iloc[0:1,0:2])

In [0]:
p_words = tokenize(random_news["text"].ascharacter())
p_words.shape

In [0]:
# how to predict...
p_reviews_vec = w2v_model.transform(p_words, aggregate_method="AVERAGE")

In [0]:
aml.leader.predict(p_reviews_vec)

In [0]:
# p17 has the max... 